# YamboWorkflow: BSE@GW

In this notebook we will compute the BSE@G0W0 absorption spectrum and excitonic eigenvalues by means of the `YamboWorkflow` workchain of the plugin.

It is possible to provide the instruction on how to automatically choose the bands to be included in the BSE Hamiltonian. 
Moreover, the workflow will perform the analysis of the QP eigenvalues in order to find the true band gap and compute the Q-related excitons. 

In [12]:
from aiida import orm, load_profile
load_profile()

from aiida.plugins import WorkflowFactory
from aiida.orm import QueryBuilder
from aiida.engine import submit

from aiida_quantumespresso.common.types import ElectronicType

import yaml

qb = QueryBuilder()
qb.append(orm.Group, filters={'label': 'hBN/bulk'}, tag='group')
qb.append(orm.StructureData, with_group='group')

loaded_structure_id = qb.all()[0][0].pk

# Read YAML file
with open("../configuration/codes_localhost.yaml", 'r') as stream:
    codes = yaml.safe_load(stream)
    
with open("../configuration/resources_localhost.yaml", 'r') as stream:
    resources = yaml.safe_load(stream)
    
options = {
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; if you want it, take ytheour previously nscf id (pk) to skip the DFT part.
    'structure_id':loaded_structure_id,
}

YamboWorkflow = WorkflowFactory('yambo.yambo.yambowf')

builder = YamboWorkflow.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides={
                'yres': "yambo": {
                    "parameters": {
                        "variables": {
                            "NGsBlkXs": [4, "Ry"],
                            "BSENGBlk": [4, "Ry"],
                            "FFTGvecs": [20, "Ry"],
                        },
                    },
                },
            }
            pseudo_family= options['pseudo_family'],
            #parent_folder=orm.load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: in that case, smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)

builder.scf.pw.metadata.options = resources

builder.nscf.pw.metadata.options = builder.scf.pw.metadata.options
builder.yres.yambo.metadata.options = builder.scf.pw.metadata.options

### Providing the inputs for the G0W0 quasiparticles simulation, to be used in the BSE step.

In the following we provide the instructions on the quasiparticles to be computed at the G0W0, and the parameters for the simulation. 
The parameters for the G0W0 run are stored in under `builder.qp` (NB: `builder.yres` is used for the final BSE calculation!).

In [11]:
QP_subset_dict= {
    'range_QP':6, #eV         , default=nscf_gap_eV*1.2
    'range_spectrum':10, #eV

}

QP_subset_dict.update({
    'split_bands':True, #default
    'extend_QP': True,  #default is False
    'consider_only':[8,9], #we explicitely compute only these bands.
    'T_smearing':1e-2, #default
    'qp_per_subset': 20,
    'parallel_runs':4,
})

builder.QP_subset_dict= orm.Dict(dict=QP_subset_dict)
builder.qp = builder.yres #we provide the same inputs for G0W0 and BSE, namely resources and settings. 

#providing the G0W0 input parameters.
params_gw = {
    'arguments': [
        'dipoles',
        'HF_and_locXC',
        'dipoles',
        'gw0',
        'ppa',],
    'variables': {
        'Chimod': 'hartree',
        'DysSolver': 'n',
        'GTermKind': 'BG',
        'NGsBlkXp': [2, 'Ry'],
        'BndsRnXp': [[1, 50], ''],
        'GbndRnge': [[1, 50], ''],
        'QPkrange': [[[1, 1, 8, 9]], ''],}}


params_gw = orm.Dict(dict=params_gw)
builder.qp.yambo.parameters = params_gw

In [12]:
builder.yres.yambo.metadata.options

{'stash': {}, 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1, 'num_cores_per_mpiproc': 1}, 'max_wallclock_seconds': 3600, 'withmpi': True, 'prepend_text': 'export OMP_NUM_THREADS=1'}

In [15]:
builder.additional_parsing = orm.List(list=['gap_','G_v','gap_GG','gap_GY','gap_GK','gap_KK','gap_GM','lowest_exciton','brightest_exciton'])

#### Submission

In [17]:
run = None

In [18]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/home/max/.conda/envs/aiida/lib/python3.9/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


uuid: 0256439a-6631-4945-887e-c5ba49ed6018 (pk: 1149) (aiida.workflows:yambo.yambo.yambowf)


# Inspecting the outputs

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance. All the outputs of YamboRestart and YamboCalculation are inherited

In [19]:
run.is_finished_ok

True

In [20]:
run.outputs.output_ywfl_parameters.get_dict()

{'QP': 1205,
 'SOC': False,
 'gap_': 5.7661077102602,
 'homo': -0.14816941235065,
 'lumo': 5.6179382979095,
 'c_max': 9,
 'q_ind': 12,
 'v_min': 8,
 'gap_GG': 7.6526085703313,
 'gap_GK': 8.4104489544958,
 'gap_GM': 7.3720397304833,
 'gap_GW': 5.7661,
 'gap_KK': 6.9361102646321,
 'homo_G': -1.7541014325738,
 'homo_K': -0.27976274271011,
 'lumo_G': 5.8985071377575,
 'lumo_K': 6.656347521922,
 'lumo_M': 5.6179382979095,
 'gap_DFT': 4.2864,
 'gap_dft': 4.286365574491,
 'nscf_pk': 1164,
 'homo_dft': 0.0,
 'lumo_dft': 4.286365574491,
 'gap_GG_dft': 6.7830101977557,
 'gap_GK_dft': 6.2494306598514,
 'gap_GM_dft': 5.5682469507486,
 'gap_KK_dft': 5.0788742178082,
 'homo_G_dft': -1.2818813762575,
 'homo_K_dft': -0.11132493421435,
 'lumo_G_dft': 5.5011288214982,
 'lumo_K_dft': 4.9675492835939,
 'lumo_M_dft': 4.286365574491,
 'lowest_exciton': 5.5418386459351,
 'brightest_exciton': 10.772728919983,
 'candidate_for_BSE': True,
 'lowest_exciton_index': 1,
 'brightest_exciton_index': 60}

We can see that, among the other, we have `gap_GW`. This is the GW gap as computed from the quasiparticle corrections computed along the BZ (and used in the BSE). 
So, the workflow analyse the merged ndb.QP and finds the true GW gap (which may be at different k-points with respect to the DFT one) and uses it in case we need to determine its Q-point (the transfer momentum between electron and hole composing our excitons).
(*v_min* (*c_max) represents the lowest(highest) valence (conduction) band used in BSE)

In [21]:
run.outputs.nscf_mapping.get_dict()

{'soc': False,
 'gap_': [[14, 14, 8, 8], [7, 7, 9, 9]],
 'gap_GG': [[1, 1, 8, 8], [1, 1, 9, 9]],
 'gap_GK': [[1, 1, 8, 8], [13, 13, 9, 9]],
 'gap_GM': [[1, 1, 8, 8], [7, 7, 9, 9]],
 'gap_KK': [[13, 13, 8, 8], [13, 13, 9, 9]],
 'homo_k': 14,
 'lumo_k': 7,
 'valence': 8,
 'gap_type': 'indirect',
 'conduction': 9,
 'nscf_gap_eV': 4.286,
 'dft_predicted': 'semiconductor/insulator',
 'number_of_kpoints': 14,
 'magnetic_calculation': False}

## How to start from already computed set of QP

It is possible also to skip the QP calculation, if you have already done it.
```python
builder.yres.yambo.QP_corrections = orm.load_node(<QP_db pk>)

bse_params = builder.yres.yambo.parameters.get_dict()
bse_params['variables']['KfnQPdb'] = "E < ./ndb.QP"

builder.yres.yambo.parameters = orm.Dict(dict=bse_params)

if not 'KfnQPdb' in builder.yres.yambo.parameters.get_dict()['variables'].keys():
    raise KeyError ("Key KfnQPdb not found in yambo parameters, try to copy.deepcopy the `bse_params` dict before to update it.") 

builder.parent_folder = orm.load_node(<previous yambo calc pk>).outputs.remote_folder

# To also reuse the computed outputs (like the em1d fragments)
builder.yres.yambo.settings = update_dict(self.ctx.yambo_inputs.yambo.settings, 'COPY_DBS', True)
```